# Tarea 4
### Juan Esteban Arboleda
### Luccas Rojas

# 1. Carga de los archivos 

In [63]:
import os
import pandas as pd
import numpy as np
import gensim
import re
import nltk
from tensorflow import keras
from gensim.models import KeyedVectors
from IPython.display import display_html
import pandas as pd

In [65]:
shakespeare = '../data/Shakespeare'
jane_austen = '../data/JaneAusten'
lovecraft = '../data/Lovecraft'

def load_books(path):
    """
    Carga los libros de un autor y los devuelve en una lista
    Params:
    ------
    path: str
        Path de la carpeta que contiene los libros
    Returns:
    -------
    books: list
        Lista de strings, cada uno representa un libro
    """
    books = []
    book_paths = os.listdir(path)
    for book_path in book_paths:
        book = open(path + '/' + book_path, 'r', encoding='utf-8')
        books.append(book.read())
        book.close()
    return books
shakespeare_books = load_books(shakespeare)
jane_austen_books = load_books(jane_austen)
lovecraft_books = load_books(lovecraft)

books = {'shakespeare': shakespeare_books, 'jane_austen': jane_austen_books, 'lovecraft': lovecraft_books}

In [66]:
def normalize(sentence:str)->str:
    """
    Normalize a sentence by lowercasing it and removing non alphanumeric characters
    Params:
    -------
        sentence: sentence to normalize
    Returns:
    --------
        sentence: normalized sentence
    """

    sentence = re.sub(r"[^(a-zA-Z0-9\s)]", " ", sentence).lower().replace("\n", " ")

    return sentence

for author in books.keys():
    for i in range(len(books[author])):
        books[author][i] = normalize(books[author][i])

In [67]:
def tokenize(document:str)->list:
    """
    Tokenize a document
    Params:
    -------
        document: document to tokenize  
    Returns:
    --------
        tokens: list of tokens
    """
    return gensim.utils.simple_preprocess(document, deacc=True)

books_list =[]
for author in books.keys():
    for book in books[author]:
        books_list.append(tokenize(book))

In [69]:
PATH_EMBEDDINGS_50 = "../models/embeddings_50.model"
embeddings_50 = gensim.models.Word2Vec(books_list, vector_size=50, window=4, min_count=2, workers=10)
if not os.path.exists(PATH_EMBEDDINGS_50):
    embeddings_50.train(books_list,total_examples=len(books_list),epochs=100)

    embeddings_50.save(PATH_EMBEDDINGS_50)
    print("Modelo creado y guardado")
else:
    embeddings_50 = gensim.models.Word2Vec.load(PATH_EMBEDDINGS_50)
    print("Modelo cargado")

Modelo creado y guardado


In [95]:
embeddings_50.wv.most_similar('cat', topn=5)

[('expressly', 0.718009889125824),
 ('coxcombs', 0.6330082416534424),
 ('slave', 0.6319321393966675),
 ('rung', 0.6313157677650452),
 ('everlasting', 0.6285082697868347)]

In [89]:
relationship = embeddings_50.wv['queen'] - embeddings_50.wv['king'] + embeddings_50.wv['man']

# Encuentra palabras más similares a la relación calculada
embeddings_50.wv.most_similar([relationship], topn=3)

[('man', 0.695233941078186),
 ('woman', 0.47210273146629333),
 ('named', 0.46514174342155457)]

In [ ]:
embeddings_100 = gensim.models.Word2Vec(books_list, size=100, window=4, min_count=2, workers=10)